# Adaptive Cards Toolkit: 02 - Dashboard Example

This notebook demonstrates how to create dashboard-style cards with statistics and indicators. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook.

In [ ]:
# First run the common utilities notebook
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import sys
import os

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import ElementFactory directly to ensure it's available
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper

## Create Statistic Containers

Let's define a function to create statistic containers with optional trend indicators.

In [ ]:
# Create a function to build statistic containers
def create_statistic_container(title, value, trend=None, trend_direction=None):
    """Create a container with a statistic and optional trend indicator.
    
    Args:
        title: The title of the statistic
        value: The value to display
        trend: Optional trend percentage 
        trend_direction: Optional direction ('up' or 'down')
        
    Returns:
        A container element with the statistic
    """
    items = [
        ElementFactory.create_text(title, is_subtle=True),
        ElementFactory.create_heading(value, level=2)
    ]
    
    if trend is not None and trend_direction is not None:
        trend_color = "good" if trend_direction == "up" else "attention"
        trend_text = f"↑ {trend}%" if trend_direction == "up" else f"↓ {trend}%"
        items.append(ElementFactory.create_important_text(trend_text, color=trend_color))
    
    return LayoutHelper.create_container(items=items, style="emphasis", spacing="medium")

## Create Statistics

Now let's create four statistics for our dashboard.

In [ ]:
# Create four statistics for our dashboard
try:
    # Create the statistics
    stat1 = create_statistic_container("Revenue", "$1.25M", "12", "up")
    stat2 = create_statistic_container("New Users", "8,742", "8", "up")
    stat3 = create_statistic_container("Churn Rate", "2.1%", "0.5", "down")
    stat4 = create_statistic_container("Avg Session", "24m 36s", "3", "up")
    
    print("Statistics created successfully!")
    print("Statistic type:", type(stat1))
    
except Exception as e:
    print(f"Error creating statistics: {e}")

## Arrange Statistics in Columns

Let's arrange the statistics in a 2x2 grid using column layouts.

In [ ]:
# Arrange statistics in two rows of two columns each
try:
    # Try different approaches to arrange in columns
    print("Creating equal columns with stats...")
    row1 = LayoutHelper.create_equal_columns([[stat1], [stat2]])
    row2 = LayoutHelper.create_equal_columns([[stat3], [stat4]])
    
    print("Rows created successfully!")
    print("Row type:", type(row1))
    
    # If available, show the structure
    if hasattr(row1, 'to_dict'):
        print("Row structure:")
        print(json.dumps(row1.to_dict(), indent=2))
        
except Exception as e:
    print(f"Error creating column layout: {e}")
    print("Trying alternative approach...")
    
    try:
        # Alternative approach
        row1 = LayoutHelper.create_column_set([
            ElementFactory.create_column(width="1", items=[stat1]),
            ElementFactory.create_column(width="1", items=[stat2])
        ])
        row2 = LayoutHelper.create_column_set([
            ElementFactory.create_column(width="1", items=[stat3]),
            ElementFactory.create_column(width="1", items=[stat4])
        ])
        print("Alternative approach successful!")
    except Exception as e2:
        print(f"Alternative approach also failed: {e2}")

## Create the Dashboard Card

Now let's create the dashboard card and add our statistics to it.

In [ ]:
# Create the dashboard card
try:
    dashboard_card = builder.create_basic_card(
        title="Q2 2024 Performance Dashboard",
        message="Key metrics for the second quarter of 2024."
    )
    
    print("Dashboard card created!")
    print("Card type:", type(dashboard_card))
    
    # Show initial card structure
    if hasattr(dashboard_card, 'to_dict'):
        print("Initial card structure:")
        initial_structure = dashboard_card.to_dict()
        print(json.dumps(initial_structure, indent=2))
    
    # Try to add our statistics rows
    print("\nAttempting to add rows to the card...")
    
    # Try different approaches based on what we find
    if hasattr(dashboard_card, 'add'):
        print("Using 'add' method...")
        dashboard_card.add(row1)
        dashboard_card.add(row2)
    elif hasattr(dashboard_card, 'add_item'):
        print("Using 'add_item' method...")
        dashboard_card.add_item(row1)
        dashboard_card.add_item(row2)
    elif hasattr(dashboard_card, 'body') and hasattr(dashboard_card.body, 'append'):
        print("Appending to 'body' attribute...")
        dashboard_card.body.append(row1)
        dashboard_card.body.append(row2)
    elif hasattr(dashboard_card, 'to_dict'):
        print("Modifying dictionary structure...")
        card_dict = dashboard_card.to_dict()
        
        if 'body' in card_dict and isinstance(card_dict['body'], list):
            row1_dict = row1.to_dict() if hasattr(row1, 'to_dict') else row1
            row2_dict = row2.to_dict() if hasattr(row2, 'to_dict') else row2
            
            card_dict['body'].append(row1_dict)
            card_dict['body'].append(row2_dict)
    else:
        print("Could not find a way to add rows to the card.")
    
    # Display the final card
    display_card(dashboard_card, "Dashboard with Statistics")
    
except Exception as e:
    print(f"Error creating dashboard: {e}")